In [10]:
# Import necessary libraries
import geopandas as gpd
import requests
from io import BytesIO
from shapely.geometry import MultiLineString, LineString

In [11]:
# URL of the GeoJSON file
url = 'https://github.com/newzealandpaul/Shipping-Lanes/blob/main/data/Shipping_Lanes_v1.geojson?raw=true'

# Fetch the GeoJSON file
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Read the GeoJSON data into a GeoDataFrame
geojson_data = BytesIO(response.content)
gdf = gpd.read_file(geojson_data)

# Display the first few rows of the GeoDataFrame
gdf.head()

,FID,OBJECTID,Type,geometry
0,0,1,Major,"MULTILINESTRING ((-34.8576 -8.13738, -34.67437..."
1,1,2,Middle,"MULTILINESTRING ((-52.14663 47.21993, -52.0640..."
2,2,3,Minor,"MULTILINESTRING ((100.43797 13.33198, 100.4257..."


In [12]:
# I need to parse the MultilineString geometries to extract the coordinates

def extract_coordinates(geometry):
    """Extract coordinates from a MultiLineString or LineString geometry."""

    

    if isinstance(geometry, MultiLineString):
        return [list(line.coords) for line in geometry.geoms]
    elif isinstance(geometry, LineString):
        return [list(geometry.coords)]
    else:
        return []

# Apply the function to extract coordinates
gdf['coordinates'] = gdf['geometry'].apply(extract_coordinates)

# Display the first few rows of the updated GeoDataFrame
gdf.head()

# Save the updated GeoDataFrame to a new GeoJSON file
#gdf.to_file("updated_shipping_lanes.geojson", driver="GeoJSON")

,FID,OBJECTID,Type,geometry,coordinates
0,0,1,Major,"MULTILINESTRING ((-34.8576 -8.13738, -34.67437...","[[(-34.85759679699993, -8.13737959499997), (-3..."
1,1,2,Middle,"MULTILINESTRING ((-52.14663 47.21993, -52.0640...","[[(-52.146626922999985, 47.21993143100008), (-..."
2,2,3,Minor,"MULTILINESTRING ((100.43797 13.33198, 100.4257...","[[(100.43797433000003, 13.331983700000023), (1..."


In [19]:
# Transform gdf so that there is a single row for each linestring in the coordinates array
# Create a new DataFrame by iterating over the rows and exploding the coordinates
exploded_data = []

for idx, row in gdf.iterrows():
    for line in row['coordinates']:
        exploded_data.append({
            'FID': row['FID'],
            'OBJECTID': row['OBJECTID'],
            'Type': row['Type'],
            'geometry': LineString(line) if isinstance(line, list) else None
        })

# Create a new GeoDataFrame from the exploded data
gdf_exploded = gpd.GeoDataFrame(exploded_data, geometry='geometry', crs=gdf.crs)

# Drop rows where geometry is None
gdf_exploded.dropna(subset=['geometry'], inplace=True)

# Display the first few rows of the exploded GeoDataFrame
gdf_exploded

,FID,OBJECTID,Type,geometry
0,0,1,Major,"LINESTRING (-34.8576 -8.13738, -34.67437 -8.07..."
1,0,1,Major,"LINESTRING (-165.50085 53.93727, -165.06269 53..."
2,0,1,Major,"LINESTRING (-166.54962 54.00381, -166.74346 54..."
3,0,1,Major,"LINESTRING (179.99999 53.13424, 179.9854 53.13..."
4,0,1,Major,"LINESTRING (174.90066 52.38317, 174.57294 52.3..."
...,...,...,...,...
234,2,3,Minor,"LINESTRING (-34.40892 -4.66871, -34.75814 -4.2..."
235,2,3,Minor,"LINESTRING (-34.40892 -4.66871, -34.74195 -4.1..."
236,2,3,Minor,"LINESTRING (138.05422 9.42472, 137.99273 9.324..."
237,2,3,Minor,"LINESTRING (-157.88571 21.23252, -157.94719 21..."


In [22]:
# Extract the coordinates from the exploded GeoDataFrame
gdf_exploded['coordinates'] = gdf_exploded['geometry'].apply(lambda geom: list(geom.coords))
# Display the first few rows of the updated exploded GeoDataFrame

# Ensure coordinates are shown to the 16th decimal place
gdf_exploded['coordinates'] = gdf_exploded['coordinates'].apply(lambda coords: [[round(coord, 16) for coord in pair] for pair in coords])

gdf_exploded.head()

,FID,OBJECTID,Type,geometry,coordinates
0,0,1,Major,"LINESTRING (-34.8576 -8.13738, -34.67437 -8.07...","[[-34.85759679699993, -8.13737959499997], [-34..."
1,0,1,Major,"LINESTRING (-165.50085 53.93727, -165.06269 53...","[[-165.50085262999997, 53.93727424300005], [-1..."
2,0,1,Major,"LINESTRING (-166.54962 54.00381, -166.74346 54...","[[-166.54962390799997, 54.003807401000074], [-..."
3,0,1,Major,"LINESTRING (179.99999 53.13424, 179.9854 53.13...","[[179.99998854000012, 53.134237075000044], [17..."
4,0,1,Major,"LINESTRING (174.90066 52.38317, 174.57294 52.3...","[[174.90065502200002, 52.38316970000005], [174..."


In [24]:
# Define a function to process each row of cooridinates
def process_coordinates(row):
    """Process coordinates to create a dictionary with FID, OBJECTID, Type, and coordinates."""
    return {
        'FID': row['FID'],
        'OBJECTID': row['OBJECTID'],
        'Type': row['Type'],
        'coordinates': row['coordinates']
    }

# Apply this function to the first row
row = gdf_exploded.iloc[0]
processed_row = process_coordinates(row)
# Display the processed row
processed_row


{'FID': np.int64(0),
 'OBJECTID': np.int64(1),
 'Type': 'Major',
 'coordinates': [[-34.85759679699993, -8.13737959499997],
  [-34.67436962399995, -8.070254808999948],
  [-34.674368303999984, -8.070249185999955],
  [-25.630149081999946, 16.957174956000074],
  [-25.483361960999957, 17.286788839000053],
  [-25.337770409999962, 17.616515286000038],
  [-25.190650972999947, 17.94635471500004],
  [-24.99966827399993, 18.37701545400006],
  [-24.806104355999935, 18.807669188000034],
  [-24.411254697999937, 19.66897126500004],
  [-24.211280570999975, 20.099627421000037],
  [-24.013673022999967, 20.53029219600006],
  [-23.81449352599998, 20.960969498000054],
  [-23.714317912999945, 21.17631406600003],
  [-23.61375363899998, 21.39166323200004],
  [-23.512802129999955, 21.60701748100007],
  [-23.41146480499998, 21.822377304000042],
  [-23.30974307599996, 22.037743189000025],
  [-23.207638347999932, 22.253115622000053],
  [-23.10515201499993, 22.468495094000048],
  [-23.002285469999947, 22.683882092